In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
NCPI0 = ""
CCPI0 = ""
CCNOPI = ""
NCCPI = ""
NCNOPI = ""

#NUMUCV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_det_var_v08_00_00_26_CV_run1_reco2_reco2.root"
NUMUCV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root"
NUMULY = "prodgenie_bnb_nu_uboone_overlay_mcc9_reco2_LYdown_v08_00_00_32_run1_reco2_reco2.root"
NUMURY = "prodgenie_bnb_nu_uboone_overlay_mcc9_reco2_LYRayleigh_v08_00_00_32_run1_reco2_reco2.root"
NUMUSCE = "prodgenie_bnb_nu_uboone_overlay_mcc9_SCE_v08_00_00_34_run1_reco2_reco2.root"
NUMUR1 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb1_reco2_reco2.root"
NUMUR2 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb2_reco2_reco2.root"
NUMUR3 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb3_reco2_reco2.root"
NUMUWA = "prodgenie_bnb_nu_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUMUWX = "prodgenie_bnb_nu_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUMUWYZ = "prodgenie_bnb_nu_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2.root"
    
DETVAR_N_V = ['LY','Rayleigh','SCE','Recomb 1','Recomb 2','Recomb 3','WireMod angle',\
             'WireMod X','WireMod YZ']

DETVAR_S_V = [NUMULY,NUMURY,NUMUSCE,NUMUR1,NUMUR2,NUMUR3,NUMUWA,NUMUWX,NUMUWYZ]
    
#CV = uproot.open(ls.ntuple_path+NUMUCV)[fold][tree]
CV = uproot.open(NUMUCV)[fold][tree]
variables = ["nu_e","run","sub","evt"]

CV  = CV.pandas.df(variables, flatten=False)

CV_E_v = CV['nu_e'].values

nCV = CV.shape[0]

for i,N in enumerate(DETVAR_N_V):
    VAR = uproot.open(ls.ntuple_path+DETVAR_S_V[i])[fold][tree]
    VAR = VAR.pandas.df(variables, flatten=False)
    
    nVAR = VAR.shape[0]
    
    #print ("entries CV : %i. Entries VAR %s : %i"%(nCV,N,nVAR))
    #continue
    
    VAR_E_v = VAR['nu_e'].values
    
    intersection = np.intersect1d(VAR_E_v,CV_E_v)
    
    nINT = float(len(intersection))
    
    #INT = pd.merge(CV, VAR, how='inner', on=['nu_e'])#,'run','sub','evt'])
    
    #nINT = float(INT.shape[0])
    
    print ('fraction of events from %15s in CV is %.02f'%(N,(nINT/nVAR)))

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

#NUECV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_det_var_v08_00_00_26_CV_run1_reco2_reco2.root"
NUECV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2.root"
NUELY = "prodgenie_bnb_intrinsic_nue_uboone_overlay_mcc9_reco2_LYdown_v08_00_00_32_run1_reco2_reco2.root"
NUEWA = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWX = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWYZ = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2.root"
    
DETVAR_N_V = ['LY','WireMod angle','WireMod X','WireMod YZ']

DETVAR_S_V = [NUELY,NUEWA,NUEWX,NUEWYZ]
    
#CV = uproot.open(ls.ntuple_path+NUECV)[fold][tree]
CV = uproot.open(NUECV)[fold][tree]
variables = ["nu_e","run","sub","evt"]

CV  = CV.pandas.df(variables+["reco_nu_vtx_sce_y"], flatten=False)

CV['identifier'] = CV['run']*100000 + CV['evt']

CV_E_v = CV['identifier'].values

nCV = CV.shape[0]

for i,N in enumerate(DETVAR_N_V):
    VAR = uproot.open(ls.ntuple_path+DETVAR_S_V[i])[fold][tree]
    VAR = VAR.pandas.df(variables+["reco_nu_vtx_sce_x"], flatten=False)
    
    VAR['identifier'] = VAR['run']*100000 + VAR['evt']
    
    nVAR = VAR.shape[0]
    
    #print ("entries CV : %i. Entries VAR %s : %i"%(nCV,N,nVAR))
    #continue
    
    VAR_E_v = VAR['identifier'].values
    
    intersection = np.intersect1d(VAR_E_v,CV_E_v)
    
    nINT = float(len(intersection))
    
    merged = pd.merge(CV,VAR, how='inner',on=["run","sub","evt"])
    
    #print (VAR.query('run == 7012'))
    
    #INT.dropna(inplace=True)
    INT = pd.merge(CV, VAR, how='inner', on=['identifier'],suffixes=('_CV', '_VAR'))#,'run','sub','evt'])
    
    nINT2 = float(INT.shape[0])
     
    print ('nINT : %i. nINT2 : %i'%(nINT,nINT2))
        
    print ('CV events : %06i. VAR events : %06i'%(nCV,nVAR))
    print ('fraction of events from %15s in CV is %.02f'%(N,(nINT/nCV)))
    print ('df CV :')
    print (CV)
    print ('df VAR :')
    print (VAR)
    print ('df INT :')
    print (INT)
    
    #break

In [ ]:
print ((VAR.query('run == 7055 and sub == 324 and evt == 16234')).shape)